In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
dados = np.arange(1, 101, 1)

In [ ]:
def gerar_janelas(lista, tam):
  '''(np.array, int) -> np.array, np.array'''
  x, y = [], []
  n = len(lista)
  for i in range(n-tam):
    janela = lista[i:i+tam]
    x.append(janela)
    y.append(lista[i+tam])
  return np.array(x), np.array(y)

In [44]:
tam_janela = 10
x, y = gerar_janelas(dados, tam_janela)
print(len(dados), "dados =>", len(x), "janelas")
for i in range(len(x)):
  print(x[i], y[i])

100 dados => 90 janelas
[ 1  2  3  4  5  6  7  8  9 10] 11
[ 2  3  4  5  6  7  8  9 10 11] 12
[ 3  4  5  6  7  8  9 10 11 12] 13
[ 4  5  6  7  8  9 10 11 12 13] 14
[ 5  6  7  8  9 10 11 12 13 14] 15
[ 6  7  8  9 10 11 12 13 14 15] 16
[ 7  8  9 10 11 12 13 14 15 16] 17
[ 8  9 10 11 12 13 14 15 16 17] 18
[ 9 10 11 12 13 14 15 16 17 18] 19
[10 11 12 13 14 15 16 17 18 19] 20
[11 12 13 14 15 16 17 18 19 20] 21
[12 13 14 15 16 17 18 19 20 21] 22
[13 14 15 16 17 18 19 20 21 22] 23
[14 15 16 17 18 19 20 21 22 23] 24
[15 16 17 18 19 20 21 22 23 24] 25
[16 17 18 19 20 21 22 23 24 25] 26
[17 18 19 20 21 22 23 24 25 26] 27
[18 19 20 21 22 23 24 25 26 27] 28
[19 20 21 22 23 24 25 26 27 28] 29
[20 21 22 23 24 25 26 27 28 29] 30
[21 22 23 24 25 26 27 28 29 30] 31
[22 23 24 25 26 27 28 29 30 31] 32
[23 24 25 26 27 28 29 30 31 32] 33
[24 25 26 27 28 29 30 31 32 33] 34
[25 26 27 28 29 30 31 32 33 34] 35
[26 27 28 29 30 31 32 33 34 35] 36
[27 28 29 30 31 32 33 34 35 36] 37
[28 29 30 31 32 33 34 35 36 37]

In [64]:
X = x.reshape((x.shape[0], tam_janela, 1))

In [83]:
model = keras.Sequential()
model.add(keras.layers.Conv1D(filters=64, kernel_size=5, activation='elu', input_shape=(tam_janela, 1)))
model.add(keras.layers.MaxPooling1D(pool_size=5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(50, activation='elu'))
model.add(keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

In [84]:
# treinar modelo
model.fit(X, y, epochs=2000, verbose=0)

In [88]:
# Exibe erro MSE para conjunto de treino
model.evaluate(X, y)

3/3 [==============================] - 0s 2ms/step - loss: 2.3876e-05


2.3875634724390693e-05

In [95]:
teste = np.arange(91, 120, 1)
x_test, y_test = gerar_janelas(teste, tam_janela)
x_test = x_test.reshape((len(x_test), tam_janela, 1))

In [101]:
print(model.predict(x_test).reshape(-1),"\n", y_test)

[100.989136 101.98757  102.98581  103.98386  104.9817   105.97935
 106.976814 107.9741   108.97121  109.96812  110.96487  111.96145
 112.957855 113.95407  114.950134 115.94604  116.94178  117.937386
 118.93281 ] 
 [101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119]


In [102]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 1ms/step - loss: 0.0015


0.0014934530481696129